In [113]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.benchmark as benchmark

torch.set_default_dtype(torch.float64)

In [126]:
def each_label(l):
    inner = [0.0] * 10
    inner[int(l)] = 1
    return inner

def format_labels(labels):
    ret_arr = []
    for i in range(len(labels)):
        ret_arr.append(each_label(labels[i]))
    #return map(each_label, labels)
    return ret_arr

def argmax(arr):
    acc = 0
    for i in range(len(arr)):
        if arr[acc] < arr[i]:
            acc = i
    return acc

In [115]:
# Load up JSON Function
import json

# Open our JSON file and load it into python
input_file = open ('../mnist-json/test-images-futhark.json')
test_images = torch.tensor(json.load(input_file))
input_file = open ('../mnist-json/test-labels-futhark.json')
test_labels = torch.tensor(format_labels(json.load(input_file)))
input_file = open ('../mnist-json/training-images-futhark.json')
training_images = torch.tensor(json.load(input_file))
input_file = open ('../mnist-json/training-labels-futhark.json')
training_labels = torch.tensor(format_labels(json.load(input_file)))

In [116]:
# Create network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Conv2d(1, 32, 3),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(1),
            nn.Linear(12 * 12 * 64, 128),
            #nn.Linear(15360, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [117]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = NeuralNetwork().to(device)

In [118]:
first_fwd = model(training_images)

In [119]:
first_fwd.size()

torch.Size([20, 10])

In [120]:
first_fwd

tensor([[2.4976e-03, 5.7604e-05, 9.4133e-05, 1.3860e-05, 1.8291e-06, 1.5349e-06,
         9.9723e-01, 9.8136e-05, 4.5356e-08, 1.0104e-05],
        [3.3367e-05, 6.3968e-09, 1.4203e-06, 4.0046e-08, 5.4766e-11, 4.3241e-09,
         9.9996e-01, 6.4559e-06, 1.1431e-10, 1.0996e-09],
        [2.8063e-02, 2.5476e-07, 1.9963e-06, 8.6563e-05, 6.4093e-06, 4.9326e-09,
         9.6663e-01, 5.2076e-03, 4.9687e-06, 3.5841e-07],
        [6.3701e-05, 2.0526e-07, 1.3393e-08, 2.0957e-08, 6.4437e-10, 6.0507e-09,
         9.9991e-01, 2.1101e-05, 8.4372e-09, 1.3051e-08],
        [3.0920e-02, 5.7860e-04, 5.2051e-05, 3.6486e-05, 1.5025e-05, 2.2232e-01,
         4.4909e-01, 2.9625e-01, 1.2766e-07, 7.3977e-04],
        [1.6836e-01, 1.2606e-05, 9.8838e-05, 1.0272e-07, 1.6224e-08, 6.9759e-07,
         8.3101e-01, 5.1571e-04, 1.3239e-09, 5.8937e-07],
        [2.4920e-01, 1.0479e-01, 1.4665e-03, 9.9933e-03, 2.0768e-02, 2.6732e-02,
         5.3742e-01, 2.5464e-02, 4.7113e-07, 2.4165e-02],
        [1.0672e-06, 3.1173

In [121]:
loss_fn = torch.nn.MSELoss()
#loss_fn(, training_labels)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
print(first_fwd.size(), training_labels.size())
# First loss
loss_fn(first_fwd, training_labels)

torch.Size([20, 10]) torch.Size([20, 10])


tensor(0.1571, grad_fn=<MseLossBackward>)

In [122]:
def train_one_epoch():
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    
    #optimizer.zero_grad()
    outputs = model(training_images)
    
    loss = loss_fn(outputs, training_labels)
    loss.backward()
    
    optimizer.step()

    return loss

In [123]:
for i in range(50):
    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch()

    # We don't need gradients on to do reporting
    model.train(False)

avg_loss

tensor(0.1400, grad_fn=<MseLossBackward>)

In [125]:
loss_fn(model(test_images), test_labels)

tensor(0.1600, grad_fn=<MseLossBackward>)

In [136]:
import torch.utils.benchmark as benchmark

# BENCHMARKING
def train_model(iterations=50):
    model = NeuralNetwork().to(device)
    for i in range(iterations):
        # Make sure gradient tracking is on, and do a pass over the data
        model.train(True)
        avg_loss = train_one_epoch()

        # We don't need gradients on to do reporting
        model.train(False)
    # Model is trained now
    # Find accuracy
    labels = list(map(argmax, test_labels))
    output = model(test_images)
    predictions = list(map(argmax, output))
    hits = 0
    for i in range(len(predictions)):
        if predictions[i] == labels[i]:
            hits += 1
    accuracy = hits / len(predictions)
    return accuracy
    

In [137]:
t0 = benchmark.Timer(
    stmt='train_model()',
    setup='from __main__ import train_model',
    globals={'NeuralNetwork': NeuralNetwork, 
             'device': device,
             'test_images': test_images,
             'test_labels': test_labels})


print(t0.timeit(2))

train_model()
setup: from __main__ import train_model
  3.15 s
  1 measurement, 2 runs , 1 thread
